

### Json Model


In [1]:
!pip install sentence-transformers==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached 

In [15]:
import pandas as pd
import numpy as np
import os

# Define the path to the folder containing the CSV files
# folder_path = r'/content/processed'
folder_path = r'./processed'

# List to hold dataframes
dfs = []

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):  # Check if the file is a CSV
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)  # Load the CSV file

        # Check if 'IsQuestion' column exists
        if 'IsQuestion' in df.columns:
            # Convert NaN values to 0 and then the column to integers
            df['IsQuestion'] = df['IsQuestion'].fillna(0).astype(int)
            dfs.append(df)  # Append the dataframe to the list

# Concatenate all dataframes in the list
combined_df = pd.concat(dfs, ignore_index=True)

# If you want to save the combined dataframe to a new CSV file
# combined_df.to_csv('combined_csv.csv', index=False)

# Show the first few rows of the combined dataframe
combined_df.head()


,Cell Data,Filename,Table Number,IsQuestion
0,"To code, please use a checkmark () for each s...",C:\Users\tates\Downloads\pdf-questionnaire-ext...,1,0
1,NOT \nTRUE,C:\Users\tates\Downloads\pdf-questionnaire-ext...,1,0
2,SOME \nTIMES,C:\Users\tates\Downloads\pdf-questionnaire-ext...,1,0
3,TRUE,C:\Users\tates\Downloads\pdf-questionnaire-ext...,1,0
4,1. I felt miserable or unhappy.,C:\Users\tates\Downloads\pdf-questionnaire-ext...,1,1


In [16]:
combined_df.shape

(1028, 4)

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(combined_df['Cell Data'].fillna(''), combined_df['IsQuestion'], test_size=0.1, random_state=42)

# Ensure X_train and X_test are lists
X_train_list = X_train.tolist()
X_test_list = X_test.tolist()

In [7]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sentence_transformers import SentenceTransformer

def create_embeddings(X):
    model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
    return np.array(model.encode(X, show_progress_bar=True))


In [18]:
from sentence_transformers import SentenceTransformer
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score

def create_embeddings(X):
    model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
    return np.array(model.encode(X, show_progress_bar=True))

# Create embeddings for training and testing data
X_train_embeddings = create_embeddings(X_train_list)
X_test_embeddings = create_embeddings(X_test_list)

# Initialize and train the XGBoost classifier
classifier = XGBClassifier(use_label_encoder=False, eval_metric='logloss', n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
classifier.fit(X_train_embeddings, y_train)

# Predict the labels for the test data
y_pred_xgb = classifier.predict(X_test_embeddings)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred_xgb))
print("\nClassification Report:\n", classification_report(y_test, y_pred_xgb))


Batches:   0%|          | 0/29 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Accuracy: 0.9902912621359223

Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      0.99        74
           1       1.00      0.97      0.98        29

    accuracy                           0.99       103
   macro avg       0.99      0.98      0.99       103
weighted avg       0.99      0.99      0.99       103



In [28]:
import pickle

with open('models/json_xgb.pkl', 'wb') as f:
    pickle.dump(classifier, f)

In [29]:
with open(r'./models/json_xgb.pkl', 'rb') as file:
            json_model = pickle.load(file)

In [30]:
def predict_text(text_list):
    # Generate embeddings for the input text
    embeddings = create_embeddings(text_list)

    # Use the trained classifier to predict
    predictions = json_model.predict(embeddings)

    return predictions

# Example usage
new_texts = ["what is your name", "i am feeling happy"]  # Replace with your actual text
predicted_labels = predict_text(new_texts)
print("Predicted Labels:", predicted_labels)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Predicted Labels: [1 1]


In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import FunctionTransformer

# Initialize the pipeline with the embedding function and XGBClassifier
model_xgb = Pipeline([
    ('embedder', FunctionTransformer(create_embeddings, validate=False)),  # Turn off validation to accept non-2D input
    ('classifier', XGBClassifier(use_label_encoder=False, eval_metric='logloss', n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42))
])

In [ ]:
import numpy as np
import random

filepaths = combined_df['Filename'].unique()
random.shuffle(filepaths)

def split_data(filepaths, n_validation, all_files):
    validation_files = filepaths[:n_validation]
    training_files = set(all_files) - set(validation_files)
    return validation_files, training_files

n_validation_files = 2

all_files = filepaths
scores = []
while len(filepaths) > 0:
    validation_files, training_files = split_data(filepaths, n_validation_files, all_files)

    validation_data = combined_df[combined_df['Filename'].isin(validation_files)]

    training_data = combined_df[combined_df['Filename'].isin(training_files)]

    model_xgb = Pipeline([
    ('embedder', FunctionTransformer(create_embeddings, validate=False)),
    ('classifier', XGBClassifier(use_label_encoder=False, eval_metric='logloss', n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)),
    ])
    model_xgb.fit(training_data['Cell Data'].tolist(), training_data['IsQuestion'])

    y_pred_xgb = model_xgb.predict(validation_data['Cell Data'].tolist())
    score = accuracy_score(validation_data['IsQuestion'], y_pred_xgb)
    scores.append(score)
    print("Accuracy:", score)
    print("\nClassification Report:\n", classification_report(validation_data['IsQuestion'], y_pred_xgb))
    filepaths = filepaths[n_validation_files:]
np.average(scores)


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Accuracy: 0.5357142857142857

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.54      0.70        28
           1       0.00      0.00      0.00         0

    accuracy                           0.54        28
   macro avg       0.50      0.27      0.35        28
weighted avg       1.00      0.54      0.70        28



c:\Users\tates\Classwork\open-source\harmony\harmony\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\tates\Classwork\open-source\harmony\harmony\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\tates\Classwork\open-source\harmony\harmony\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

...

Accuracy: 0.9877300613496932

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.97      0.99        77
           1       0.98      1.00      0.99        86

    accuracy                           0.99       163
   macro avg       0.99      0.99      0.99       163
weighted avg       0.99      0.99      0.99       163



Batches:   0%|          | 0/30 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Accuracy: 0.968421052631579

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.94      0.97        51
           1       0.94      1.00      0.97        44

    accuracy                           0.97        95
   macro avg       0.97      0.97      0.97        95
weighted avg       0.97      0.97      0.97        95



0.8918671419338657

# text_data

In [35]:
import pandas as pd
import numpy as np
import os

# Define the path to the folder containing the CSV files
# folder_path = r'/content/text_processed'
folder_path = r'./text_processed'


# List to hold dataframes
dfs = []
filenames = []  # List to store filenames

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):  # Check if the file is a CSV
        # print(filename)  # Print the filename
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)  # Load the CSV file
        # Add a 'Filename' column filled with the current filename
        df['Filename'] = filename
        # Check if 'IsQuestion' column exists
        if 'IsQuestion' in df.columns:
            print(filename)
            # Convert NaN values to 0 and then the column to integers
            df['IsQuestion'] = df['IsQuestion'].fillna(0).astype(int)
            dfs.append(df)  # Append the dataframe to the list
            filenames.append(filename)  # Append filename to the list

# Concatenate all dataframes in the list
combined_df = pd.concat(dfs, ignore_index=True)

# If you want to save the combined dataframe to a new CSV file
# combined_df.to_csv('combined_csv.csv', index=False)

# Show the first few rows of the combined dataframe
combined_df.head()


000_mfqchildselfreportshort.csv
001_patienthealthquestionnaire.csv
002_bhrcsparentreportysi.csv
003_bhrcsselfreporteddeliberartese.csv
004_newyorklongitudinalstudybythom.csv
005_beckdepressioninventorybdi.csv
006_apadsm5severitymeasurefordepre.csv
007_bhrcsselfreportedscared.csv
008_gad7anxietyupdated0.csv
009_validationoftheportugueseversi.csv
010_patienthealthquestionnairephq9.csv
011_bhrcsparentreportsocialaptitud.csv
012_hrcw2protocolopsicoconfmenor18.csv
013_hrcw2protocolopsicoadultos.csv
014_testedeansiedadegad7dranadimme.csv
015_hrcw1dom.csv
016_hrcw2protocolodomiciliarmenor1.csv
017_validationoftheportugueseversi.csv
018_scaredformparentandchildversio.csv
019_bhrcsparentreportsocialcohesio.csv
020_hrcw2protocolopsicomenor18.csv
021_sdqenglishukpt417single.csv
022_cesd.csv
023_hrcw1psico.csv
024_bhrcsselfreportedmfq.csv
025_phq9questionnaire.csv
026_sf36.csv
027_hrcw2protocolodomiciliaradulto.csv
028_mfqchildselfreportlong.csv
029_bhrcsselfreportedsdq.csv
030_bhrcsselfreportedwa

,Text,IsQuestion,Filename,comment,Note,Unnamed: 2
0,NaN,0,000_mfqchildselfreportshort.csv,NaN,NaN,NaN
1,Child Self-Report,0,000_mfqchildselfreportshort.csv,NaN,NaN,NaN
2,NaN,0,000_mfqchildselfreportshort.csv,NaN,NaN,NaN
3,NaN,0,000_mfqchildselfreportshort.csv,NaN,NaN,NaN
4,NaN,0,000_mfqchildselfreportshort.csv,NaN,NaN,NaN


In [36]:
combined_df.isna().sum()

Text           91967
IsQuestion         0
Filename           0
comment       175777
Note          176405
Unnamed: 2    176383
dtype: int64

In [37]:
combined_df.shape

(176429, 6)

In [38]:
combined_df["IsQuestion"].value_counts()

IsQuestion
0    165065
1     11364
Name: count, dtype: int64

In [41]:
combined_df["Text"] = combined_df["Text"].fillna('')

In [42]:
import pandas as pd

df = combined_df[['Text', 'IsQuestion']]

# Function to combine rows of 'Text' column
def combine_text_rows(df):
    combined_text = []
    combined_other = []
    current_text = ''
    current_other = ''

    for index, row in df.iterrows():
        if row['Text'] != '':
            current_text += str(row['Text']) + ' '
            current_other = row['IsQuestion']
        else:
            if current_text != '':
                combined_text.append(current_text.strip())
                combined_other.append(current_other)
                current_text = ''

    # If there's remaining text after the loop ends
    if current_text != '':
        combined_text.append(current_text.strip())
        combined_other.append(current_other)

    combined_df = pd.DataFrame({'Text': combined_text, 'IsQuestion': combined_other})
    return combined_df

# Call the function to combine rows
processed_df = combine_text_rows(df)
print(processed_df)


                                                    Text  IsQuestion
0                                      Child Self-Report           0
1         MOOD AND FEELINGS QUESTIONNAIRE: Short Version           0
2      This form is about how you might have been fee...           0
3      For each question, please check () how you ha...           0
4      If a sentence was not true about you, check NO...           0
...                                                  ...         ...
52355  Q34. Por favor, marque com um X no número do q...           0
52356  (   ) Quadro 1 (   ) Quadro 2 (   ) Quadro 4 (...           0
52357  Q35. Por favor, marque com um X no número do q...           0
52358  (   ) Quadro 1 (   ) Quadro 2 (   ) Quadro 4 (...           0
52359  ÁLCOOL CIGARRO E SUBSTÂNCIAS RELAÇÕES SEXUAIS ...           0

[52360 rows x 2 columns]


In [10]:
processed_df.to_csv("processed.csv", index=False)

In [11]:
processed_df.shape

(52360, 2)

In [12]:
processed_df["IsQuestion"].value_counts()

IsQuestion
0    44889
1     7471
Name: count, dtype: int64

In [44]:
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score

In [46]:
#for undersampling
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming combined_df is your DataFrame
# Separate the classes
class_0 = processed_df[processed_df['IsQuestion'] == 0]
class_1 = processed_df[processed_df['IsQuestion'] == 1]

# Sample 30% of class 0 without replacement
class_0_sampled = class_0.sample(frac=0.3, random_state=42)

# Combine the sampled class 0 data with all of class 1 data
combined_sampled_df = pd.concat([class_0_sampled, class_1])

# Now you split the combined_sampled_df into training and testing sets
X_train_us, X_test_us, y_train_us, y_test_us = train_test_split(
    combined_sampled_df['Text'],
    combined_sampled_df['IsQuestion'],
    test_size=0.1,
    random_state=42
)

# Ensure X_train and X_test are lists if needed
X_train_list_us = X_train_us.tolist()
X_test_list_us = X_test_us.tolist()

In [ ]:
# import pickle

# with open('models/us_data.pkl', 'wb') as f:
#     pickle.dump(combined_sampled_df, f)

In [ ]:
base_learners = [('rf_model', RandomForestClassifier(criterion = 'entropy', max_depth = 10, max_features = 'sqrt',
                                                     max_leaf_nodes = 8, min_samples_leaf = 5, min_samples_split = 2,
                                                     n_estimators = 50, random_state = 10)),
                 ('KNN_model', KNeighborsClassifier(n_neighbors = 17, metric = 'euclidean')),
                 ('NB_model', GaussianNB())]

In [48]:
X_train_embeddings = create_embeddings(X_train_list_us)
X_test_embeddings = create_embeddings(X_test_list_us)

Batches:   0%|          | 0/589 [00:00<?, ?it/s]

Batches:   0%|          | 0/66 [00:00<?, ?it/s]

In [49]:
stack_model = StackingClassifier(estimators = base_learners, final_estimator = GaussianNB())

# Train the model
stack_model.fit(X_train_embeddings, y_train_us)

# Predict the labels for the test data
y_pred_stack_us = stack_model.predict(X_test_embeddings)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test_us, y_pred_stack_us))
print("\nClassification Report:\n", classification_report(y_test_us, y_pred_stack_us))

Accuracy: 0.8051575931232091

Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.79      0.84      1342
           1       0.69      0.83      0.75       752

    accuracy                           0.81      2094
   macro avg       0.79      0.81      0.80      2094
weighted avg       0.82      0.81      0.81      2094



In [50]:
import pickle

with open('models/stack_model_with_undersampling.pkl', 'wb') as f:
    pickle.dump(stack_model, f)

In [15]:
import numpy as np
from sklearn.utils import class_weight

class_weights = list(class_weight.compute_class_weight(class_weight= 'balanced',
                                             classes = np.unique(combined_sampled_df['IsQuestion']),
                                             y = combined_sampled_df['IsQuestion']))

#df['Product'].value_counts()

class_weights.sort()

class_weights

weightsus={}


for index, weight in enumerate(class_weights) :
    weightsus[index]=weight


In [16]:
weightsus

{0: 0.7773817479765353, 1: 1.4012849685450408}

In [ ]:
import numpy as np
from sklearn.utils import class_weight

class_weights = list(class_weight.compute_class_weight(class_weight= 'balanced',
                                             classes = np.unique(processed_df['IsQuestion']),
                                             y = processed_df['IsQuestion']))

#df['Product'].value_counts()

class_weights.sort()

class_weights

weights={}


for index, weight in enumerate(class_weights) :
    weights[index]=weight

In [ ]:
y_pred = model.predict(test_embeddings)

threshold = 0.75
y_pred_labels = (y_pred > threshold).astype(int)

from sklearn.metrics import classification_report, accuracy_score

classification= classification_report(y_test, y_pred_labels)
print(classification)

164/164 [==============================] - 2s 11ms/step
              precision    recall  f1-score   support

           0       0.97      0.90      0.93      4488
           1       0.58      0.82      0.68       749

    accuracy                           0.89      5237
   macro avg       0.77      0.86      0.81      5237
weighted avg       0.91      0.89      0.90      5237



In [ ]:
# Pytorch

In [20]:
import pandas as pd
import numpy as np
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from torch import nn, optim

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        inputs = self.tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors="pt")
        label = torch.tensor(self.labels[idx], dtype=torch.float)
        return inputs, label

train_dataset = TextDataset(X_train_list_us, y_train_us.tolist(), tokenizer)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

test_dataset = TextDataset(X_test_list_us, y_test_us.tolist(), tokenizer)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [4]:
from transformers import AutoTokenizer, AutoModel
from torch import nn, optim

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
class LSTMClassifier(nn.Module):
    def __init__(self, hidden_dim, output_dim):
        super(LSTMClassifier, self).__init__()
        self.encoder = AutoModel.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
        for param in self.encoder.parameters():
            param.requires_grad = False
        self.lstm = nn.LSTM(self.encoder.config.hidden_size, hidden_dim, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, input_ids, attention_mask=None):
        with torch.no_grad():
            embedded = self.encoder(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state
        _, (hidden, _) = self.lstm(embedded)
        output = self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        return output

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# This prints out the device being used.
print("Using device:", device)
model = LSTMClassifier(128, 1).to(device)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training loop
model.train()
for epoch in range(3):  # Example: 10 epochs
    for inputs, labels in train_loader:
        input_ids = inputs['input_ids'].squeeze(1).to(device)
        attention_mask = inputs['attention_mask'].squeeze(1).to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask).squeeze()
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

Using device: cuda
Epoch 1, Loss: 0.300152450799942
Epoch 2, Loss: 0.14730721712112427
Epoch 3, Loss: 0.22678813338279724


In [25]:
from sklearn.metrics import accuracy_score

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Set model to evaluation mode
model.eval()

# Store predictions and actual labels
all_predictions = []
all_labels = []

# No gradient calculation needed for evaluation
with torch.no_grad():
    for inputs, labels in test_loader:
        # Move inputs to the correct device
        input_ids = inputs['input_ids'].squeeze(1).to(device)
        attention_mask = inputs['attention_mask'].squeeze(1).to(device)

        # Get model predictions
        outputs = model(input_ids, attention_mask).squeeze()

        # Apply sigmoid to output to get prediction probabilities
        predictions = sigmoid(outputs.cpu().numpy())
        predicted_labels = [1 if prediction > 0.5 else 0 for prediction in predictions]  # Using 0.5 as threshold

        # Store predictions and actual labels
        all_predictions.extend(predicted_labels)
        all_labels.extend(labels.cpu().numpy())

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_predictions)
print(f"Accuracy: {accuracy}")


Accuracy: 0.876313276026743


In [28]:
classification= classification_report(all_labels, all_predictions)
print(classification)

              precision    recall  f1-score   support

         0.0       0.94      0.86      0.90      1342
         1.0       0.79      0.90      0.84       752

    accuracy                           0.88      2094
   macro avg       0.86      0.88      0.87      2094
weighted avg       0.88      0.88      0.88      2094



In [33]:
#torch.save(model, 'lstm_model.pth')
torch.save(model.state_dict(), "model.pth")

In [21]:
import torch
model  = LSTMClassifier(128, 1)
model.load_state_dict(torch.load("models/model.pth", map_location=torch.device('cpu')))
model.eval()

LSTMClassifier(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(250037, 384, padding_idx=0)
      (position_embeddings): Embedding(512, 384)
      (token_type_embeddings): Embedding(2, 384)
      (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (LayerNorm): LayerNorm((384,), eps=1e-12, elementwis

In [22]:
def predict_new_text(model, text, tokenizer, device):
    # Tokenize the input text and convert to a tensor
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # Get model predictions
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)

    # Apply sigmoid function to get the probability
    probabilities = torch.sigmoid(outputs).squeeze()

    # Convert to numpy and return
    return probabilities.cpu().numpy()

# Example usage
new_text = "adb"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Ensure model is on the correct device
probabilities = predict_new_text(model, new_text, tokenizer, device)

# Assuming binary classification, decide the class based on a threshold
threshold = 0.4
predicted_class = int(probabilities > threshold)
print(f"Predicted class: {predicted_class}, Probability: {probabilities}")


Predicted class: 0, Probability: 0.019351650029420853
